In [2]:
!pip install PyWavelets
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Bidirectional, LSTM, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from scipy.io import loadmat, savemat
from scipy.signal import butter, filtfilt
import os
from google.colab import drive
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.1 MB/s eta 0:00:00


In [7]:
# Mount Google Drive
drive.mount('/content/drive')

# Define input and output folders
input_folder = "/content/drive/MyDrive/EEG Dataset"
processed_folder = os.path.join(input_folder, "Processed_CNN_BiLSTM")
model_save_folder = os.path.join(input_folder, "Models_CNN_BiLSTM")
os.makedirs(processed_folder, exist_ok=True)
os.makedirs(model_save_folder, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Band-pass filter function (0.5-70 Hz Butterworth filter)
def bandpass_filter(signal, lowcut=0.5, highcut=70.0, fs=173.61, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

# Function to extract DWT-based entropy features
def extract_dwt_entropy_features(signal, wavelet='db4', level=5):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = []
    for coeff in coeffs:
        features.append(np.mean(np.abs(coeff)))  # Approximate Entropy
        features.append(np.std(coeff))  # Standard Deviation
        features.append(np.sqrt(np.mean(coeff**2)))  # RMS
        features.append(np.mean(np.cumsum(coeff)))  # Hurst Exponent
    return np.array(features)

In [59]:
# Load and preprocess data for all folders
all_features, all_labels = [], []
folder_map = {'F': 0, 'N': 1, 'S': 2, 'Z': 3, 'O': 4}

for folder in folder_map.keys():
    folder_path = os.path.join(input_folder, folder)
    if not os.path.exists(folder_path):
        print(f"Warning: {folder_path} not found. Skipping...")
        continue

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        signal = np.loadtxt(file_path)

        # Apply band-pass filter
        filtered_signal = bandpass_filter(signal)

        # Normalize using Z-score
        scaler = StandardScaler()
        normalized_signal = scaler.fit_transform(filtered_signal.reshape(-1, 1)).flatten()

        # Extract features
        dwt_features = extract_dwt_entropy_features(normalized_signal)

        # Store features and labels
        all_features.append(dwt_features)
        all_labels.append(folder_map[folder])

# Convert to numpy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Perform SVM-RFE feature selection on the full dataset
svm = SVC(kernel='linear')
rfe = RFE(svm, n_features_to_select=12)
X_selected = rfe.fit_transform(X, y)

# Save combined processed data
save_path = os.path.join(processed_folder, "processed_combined.mat")
savemat(save_path, {"features": X_selected, "labels": y})
print(f"Processed combined EEG data saved at {save_path}")

Processed combined EEG data saved at /content/drive/MyDrive/EEG Dataset/Processed_CNN_BiLSTM/processed_combined.mat


In [60]:
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42, stratify=y)

# Define CNN-BiLSTM model
def create_cnn_bilstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=(input_shape, 1)),
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32, return_sequences=False)),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Reshape input for CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train model
model = create_cnn_bilstm_model(input_shape=X_train.shape[1], num_classes=len(np.unique(y)))
model.summary()



Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_39 (Conv1D)                   │ (None, 12, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 12, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_37 (MaxPooling1D)      │ (None, 6, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_40 (Conv1D)                   │ (None, 6, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 6, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_38 (MaxPooling1D)      │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_41 (Conv1D)                   │ (None, 3, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, 3, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_39 (MaxPooling1D)      │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_22 (Bidirectional)     │ (None, 1, 128)              │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_23 (Bidirectional)     │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_14 (Flatten)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 339,845 (1.30 MB)

 Trainable params: 338,949 (1.29 MB)

 Non-trainable params: 896 (3.50 KB)

In [44]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=64,
    verbose=1
)

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9701 - loss: 0.0872 - val_accuracy: 0.4700 - val_loss: 2.5517
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9733 - loss: 0.0793 - val_accuracy: 0.5900 - val_loss: 1.7711
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.9578 - loss: 0.0949 - val_accuracy: 0.6000 - val_loss: 1.5447
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.9648 - loss: 0.0790 - val_accuracy: 0.7000 - val_loss: 1.3376
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.9676 - loss: 0.1100 - val_accuracy: 0.7900 - val_loss: 0.7951
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9341 - loss: 0.1869 - val_accuracy: 0.7700 - val_loss: 0.8058
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.9297 - loss: 0.1529 - val_accuracy: 0.7900 - val_loss: 0.6213
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9422 - loss: 0.1491 - val_accuracy: 0.8100 - val_loss

In [45]:
# Evaluate model
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save trained model
model_save_path = os.path.join(model_save_folder, "EEG_CNN_BiLSTM_Model.keras")
model.save(model_save_path)
print(f"Model trained and saved successfully at {model_save_path}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

Model Accuracy: 0.8100

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.80      0.74        20
           1       0.79      0.75      0.77        20
           2       0.94      0.80      0.86        20
           3       0.81      0.85      0.83        20
           4       0.85      0.85      0.85        20

    accuracy                           0.81       100
   macro avg       0.82      0.81      0.81       100
weighted avg       0.82      0.81      0.81       100

Model trained and saved successfully at /content/drive/MyDrive/EEG Dataset/Models_CNN_BiLSTM/EEG_CNN_BiLSTM_Model.keras


In [63]:
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=300,
    batch_size=64,
    verbose=1
)
# Evaluate model
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save trained model
model_save_path = os.path.join(model_save_folder, "EEG_CNN_BiLSTM_Model_3.keras")
model.save(model_save_path)
print(f"Model trained and saved successfully at {model_save_path}")


Epoch 1/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 1.0000 - loss: 0.0065 - val_accuracy: 0.7900 - val_loss: 1.0349
Epoch 2/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.9957 - loss: 0.0154 - val_accuracy: 0.7700 - val_loss: 0.9747
Epoch 3/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 1.0000 - loss: 0.0083 - val_accuracy: 0.7900 - val_loss: 0.8808
Epoch 4/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.0086 - val_accuracy: 0.8400 - val_loss: 0.7526
Epoch 5/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 1.0000 - loss: 0.0056 - val_accuracy: 0.8500 - val_loss: 0.7184
Epoch 6/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.8500 - val_loss: 0.7143
Epoch 7/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 1.0000 - loss: 0.0056 - val_accuracy: 0.8600 - val_loss: 0.7197
Epoch 8/300
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 0.0063 - val_accuracy: 0.8600 - val_loss

In [54]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Bidirectional, LSTM, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from scipy.io import savemat
from scipy.signal import butter, filtfilt
import os
from google.colab import drive
import pywt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score


# Band-pass filter function (0.5-70 Hz Butterworth filter)
def bandpass_filter(signal, lowcut=0.5, highcut=70.0, fs=173.61, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

# Function to extract DWT-based entropy features
def extract_dwt_entropy_features(signal, wavelet='db4', level=5):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = []
    for coeff in coeffs:
        features.append(np.mean(np.abs(coeff)))  # Approximate Entropy
        features.append(np.std(coeff))  # Standard Deviation
        features.append(np.sqrt(np.mean(coeff**2)))  # RMS
        features.append(np.mean(np.cumsum(coeff)))  # Hurst Exponent
    return np.array(features)

# Load and preprocess data for all folders
all_features, all_labels = [], []
folder_map = {'F': 0, 'N': 1, 'S': 2, 'Z': 3, 'O': 4}

for folder in folder_map.keys():
    folder_path = os.path.join(input_folder, folder)
    if not os.path.exists(folder_path):
        print(f"Warning: {folder_path} not found. Skipping...")
        continue

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        signal = np.loadtxt(file_path)

        # Apply band-pass filter
        filtered_signal = bandpass_filter(signal)

        # Normalize using Z-score
        scaler = StandardScaler()
        normalized_signal = scaler.fit_transform(filtered_signal.reshape(-1, 1)).flatten()

        # Extract features
        dwt_features = extract_dwt_entropy_features(normalized_signal)

        # Store features and labels
        all_features.append(dwt_features)
        all_labels.append(folder_map[folder])

# Convert to numpy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Perform SVM-RFE feature selection on the full dataset
svm = SVC(kernel='linear')
rfe = RFE(svm, n_features_to_select=20)  # Increased features from 12 to 20
X_selected = rfe.fit_transform(X, y)

# Save combined processed data
save_path = os.path.join(processed_folder, "processed_combined.mat")
savemat(save_path, {"features": X_selected, "labels": y})
print(f"Processed combined EEG data saved at {save_path}")

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42, stratify=y)

# Compute class weights for imbalanced data
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Define CNN-BiLSTM model
def create_cnn_bilstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=(input_shape, 1)),
        Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Bidirectional(LSTM(128, return_sequences=True)),
        Bidirectional(LSTM(64, return_sequences=False)),
        Flatten(),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Reshape input for CNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train model
model = create_cnn_bilstm_model(input_shape=X_train.shape[1], num_classes=len(np.unique(y)))
model.summary()

model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=64,
    class_weight=class_weight_dict,
    verbose=1
)


Processed combined EEG data saved at /content/drive/MyDrive/EEG Dataset/Processed_CNN_BiLSTM/processed_combined.mat


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_27 (Conv1D)                   │ (None, 20, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 20, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_25 (MaxPooling1D)      │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_28 (Conv1D)                   │ (None, 10, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_13               │ (None, 10, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_26 (MaxPooling1D)      │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_29 (Conv1D)                   │ (None, 5, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 5, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_27 (MaxPooling1D)      │ (None, 2, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_14 (Bidirectional)     │ (None, 2, 256)              │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_15 (Bidirectional)     │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 701,061 (2.67 MB)

 Trainable params: 700,165 (2.67 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 194ms/step - accuracy: 0.4190 - loss: 1.5445 - val_accuracy: 0.3400 - val_loss: 1.6150
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6556 - loss: 1.1961 - val_accuracy: 0.3200 - val_loss: 1.6031
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6988 - loss: 0.8942 - val_accuracy: 0.2000 - val_loss: 1.5913
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.7404 - loss: 0.6671 - val_accuracy: 0.2400 - val_loss: 1.5906
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7724 - loss: 0.5324 - val_accuracy: 0.4200 - val_loss: 1.5988
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7808 - loss: 0.4804 - val_accuracy: 0.3100 - val_loss: 1.5990
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7925 - loss: 0.5072 - val_accuracy: 0.2400 - val_loss: 1.6054
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.8130 - loss: 0.4469 - val_accuracy: 0.3000 - val_los

In [55]:
# Evaluate model
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save trained model
model_save_path = os.path.join(model_save_folder, "EEG_CNN_BiLSTM_Model_2.keras")
model.save(model_save_path)
print(f"Model trained and saved successfully at {model_save_path}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 690ms/step

Model Accuracy: 0.8000

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.60      0.63        20
           1       0.80      0.80      0.80        20
           2       0.80      0.80      0.80        20
           3       0.89      0.80      0.84        20
           4       0.83      1.00      0.91        20

    accuracy                           0.80       100
   macro avg       0.80      0.80      0.80       100
weighted avg       0.80      0.80      0.80       100

Model trained and saved successfully at /content/drive/MyDrive/EEG Dataset/Models_CNN_BiLSTM/EEG_CNN_BiLSTM_Model_2.keras
